# Set up OpenAI and start with a system prompt

In [1]:
import json
import os

from openai import OpenAI

In [2]:
EXAMPLES_DIR = 'examples'
os.makedirs(EXAMPLES_DIR, exist_ok=True)

In [3]:
# Load your API key from an environment variable or secret management servic
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=open('../key.txt', 'r').read().strip('\n'),
)

In [4]:
system_prompt = """# Instructions:
You are an autocomplete system for Python.

The user may
1. Describe code, either directly or in a Python comment, in which case you implement the code.
2. Write code, in which case you autocomplete the code.

## Example 1:

[INPUT]
```python
# Comment describing the code
```
[/INPUT]

[ANSWER]
```python
# Comment describing the code
<code>
```
[/ANSWER]

## Example 2:

[INPUT]
Description of the code
[/INPUT]

[ANSWER]
```python
<code>
```
[/ANSWER]

## Example 3:

[INPUT]
```python
<incomplete code snippet>
```
[/INPUT]

[ANSWER]
```python
<incomplete code snippet><code>
```
[/ANSWER]

# Rules:
- Your entire response must be valid python code.
- Any comments or notes must must be made in Python code block, i.e. ```python ... ```.
- You must not add any additional comments or notes outside of ```python ... ``` blocks, i.e. your entire response must only consist of one Python code block.
- You must end your response after ending the python code block with "```".
"""

In [5]:
def add_message(messages: list[str], role: str, message: str | None = None) -> list[str]:
    # If the user is the assistant, generate a response
    if role == "assistant" and message is None:
        chat_completion = client.chat.completions.create(messages=messages, model="gpt-3.5-turbo")
        message = chat_completion.choices[0].message.content

    # Add the message to the messages list
    messages.append({
        "role": role,
        "content": message,
    })

    return messages

# Generate a response to an example prompt

In [6]:
user_input = """Implement an algorithm that reads a csv file, splits it into chunks based on the first letter in the 'Name' column and stores the chunks into separate csv files."""

In [7]:
messages = []

In [8]:
messages = add_message(messages, "system", system_prompt)
messages = add_message(messages, "user", user_input)

In [9]:
messages = add_message(messages, "assistant")

# Examine the output and store the conversation and a comment

In [19]:
print(messages[-1]['content'])

```python
import pandas as pd

def split_csv_file(input_file, output_dir):
    # Read the CSV file into a DataFrame
    data = pd.read_csv(input_file)
    
    # Group the DataFrame by first letter of 'Name' column
    grouped_data = data.groupby(data['Name'].str[0])
    
    # Save each group into a separate CSV file
    for letter, group in grouped_data:
        output_file = f"{output_dir}/{letter}.csv"
        group.to_csv(output_file, index=False)
```
```


In [20]:
import pandas as pd

def split_csv_file(input_file, output_dir):
    # Read the CSV file into a DataFrame
    data = pd.read_csv(input_file)
    
    # Group the DataFrame by first letter of 'Name' column
    grouped_data = data.groupby(data['Name'].str[0])
    
    # Save each group into a separate CSV file
    for letter, group in grouped_data:
        output_file = f"{output_dir}/{letter}.csv"
        group.to_csv(output_file, index=False)


In [22]:
split_csv_file('names.csv', 'output')

In [32]:
comment = """
+ Correct implementation of the algorithm
- Does not check if the output directory exists and results in an `OSError: Cannot save file into a non-existent directory: 'output'`
- Additional ``` at the end of the response
"""

title = "Split a CSV file into chunks based on the first letter in the 'Name' column"

In [33]:
# Remove any previous comments
messages = [message for message in messages if message['role'] != 'comment']

# Add the comment to the messages list
messages.append({
    "role": "comment",
    "content": comment,
})

In [34]:
# Save the messages to a file
with open(os.path.join(EXAMPLES_DIR, f"{title}.json"), "w") as f:
    json.dump(messages, f, indent=4)